In [ ]:
import torch # type: ignore
from transformers import AutoTokenizer, AutoModelForCausalLM # type: ignore
from transformers import BitsAndBytesConfig # type: ignore
from peft import PeftModel # type: ignore

from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template
from src.core.solvers.l2m import L2MSolver
from src.core.solvers.self_verification import SelfVerifier
from src.core.solvers.simplified_enhanced_solver import SimplifiedEnhancedSolver

In [ ]:
class AdaptiveSolver:
    def __init__(self, model, tokenizer):
        """
        初始化自适应求解器
        Args:
            model: 已初始化的模型实例
            tokenizer: 已初始化的tokenizer实例
        """
        self.model = model
        self.tokenizer = tokenizer
        
        # 初始化各个求解器
        self.l2m_solver = L2MSolver(model, tokenizer)
        self.verifier = SelfVerifier(model, tokenizer)
        self.enhanced_solver = SimplifiedEnhancedSolver(self.l2m_solver, self.verifier)
        
        # 添加评估示例
        self.complexity_examples = """
        示例问题及其复杂度评分：

        问题1：今天的天气怎么样？
        复杂度：0.1
        原因：简单的事实性问题，可直接回答。

        问题2：请解释DNA的双螺旋结构。
        复杂度：0.5
        原因：需要分步骤解释，包含多个知识点。

        问题3：分析人工智能对未来社会的影响。
        复杂度：0.9
        原因：需要多角度分析，涉及复杂因果关系。
        """
        
    def evaluate_complexity(self, question):
        """评估问题复杂度"""
        dialogue = [
            {
                "role": "system",
                "content": "你是一个专业的问题复杂度评估专家。请根据示例评估问题的复杂度。"
            },
            {
                "role": "user",
                "content": f"""{self.complexity_examples}

请评估这个问题的复杂度：{question}

评分标准：
0.0-0.3：简单问题，可直接回答
0.3-0.7：中等复杂度，需要分步骤解答
0.7-1.0：高度复杂，需要详细分析

请只输出一个0-1之间的数字分数。"""
            }
        ]
        
        prompt = apply_chat_template(dialogue)
        response = generate_response(
            self.model, 
            self.tokenizer, 
            prompt,
            max_new_tokens=8,
            temperature=0.1
        ).strip()
        
        try:
            score = float(response)
            return min(max(score, 0.0), 1.0)
        except ValueError:
            return 0.5

    def solve(self, question):
        """根据问题复杂度选择合适的求解方法"""
        complexity = self.evaluate_complexity(question)
        print(f"问题复杂度评分: {complexity}")
        
        if complexity < 0.3:
            print("使用直接回答...")
            dialogue = [
                {
                    "role": "system",
                    "content": "你是一个专业的问答助手。请直接、简洁地回答问题。"
                },
                {
                    "role": "user",
                    "content": question
                }
            ]
            prompt = apply_chat_template(dialogue)
            answer = generate_response(
                self.model,
                self.tokenizer,
                prompt,
                max_new_tokens=512,
                temperature=0.7
            )
            
            return {
                "method": "direct",
                "complexity": complexity,
                "answer": answer
            }
            
        elif complexity < 0.7:
            print("使用L2M方法...")
            result = self.l2m_solver.solve(question)
            return {
                "method": "l2m",
                "complexity": complexity,
                "answer": result["final_answer"],
                "sub_solutions": result["sub_solutions"]
            }
            
        else:
            print("使用增强求解方法...")
            result = self.enhanced_solver.solve_complex_question(question)
            return {
                "method": "enhanced",
                "complexity": complexity,
                "answer": result["final_answer"],
                "sub_solutions": result["sub_solutions"],
                "verification": result["verification"]
            }

In [ ]:
# 使用示例
if __name__ == "__main__":
    # 初始化模型和tokenizer
    model_path = "google/gemma-2-9b"
    cache_dir = "/root/autodl-tmp/gemma"
    # lora_path = "/root/autodl-tmp/models/stage1/gemma-base-zh-final"
    lora_path = None
    
    # 初始化基础组件
    model, tokenizer = initialize_model_and_tokenizer(
        model_path=model_path,
        cache_dir=cache_dir,
        lora_path=lora_path,
        use_quantization=True
    )
    
    # 初始化自适应求解器
    solver = AdaptiveSolver(model, tokenizer)
    
    # 测试不同复杂度的问题
    questions = [
        "今天星期几？",  # 简单
        "请解释光合作用的过程。",  # 中等
        "分析人工智能对未来社会的影响。"  # 复杂
    ]
    
    for q in questions:
        print(f"\n处理问题: {q}")
        result = solver.solve(q)
        print(f"使用方法: {result['method']}")
        print(f"复杂度评分: {result['complexity']}")
        print("答案:", result['answer'])